# Aula 4 - Parte Prática - Actor-Critic (A2C)

## Introdução

Nesse quarto notebook vamos estudar um dos algoritmos mais implementados da família dos *Policy Gradients*. Ao implementar o A2C, você terá contato com importantes conceitos utilizados em *Deep RL*. Em particular, utilizaremos pela primeira vez no curso redes neurais com *features* compartilhadas e ambientes vetorizados (i.e., paralelizados) para coleta de dados mais eficiente.


### Objetivos:

- Familiarizar-se com os componentes *Actor* e *Critic*
- Entender o papel da função Valor na estimativa truncada dos retornos
- Ter um primeiro contato com truques de implementação tipicamente utilizados e RL


### Imports

> **Atenção:** não se esqueça de executar todos os `imports` necessários antes prosseguir com o tutorial.

In [1]:
import logging
from pprint import pprint

import gym
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

from utils.agent import RLAgent, RandomAgent
from utils.memory import OnPolicyReplay
from utils.networks import build_actor_critic_network
import utils.runner
from utils.viz import *


tf.get_logger().setLevel("ERROR")     # ignore TensorFlow warnings
gym.logger.set_level(logging.ERROR)   # ignore OpenAI Gym warnings

## 0. LunarLander-v2

Para o notebook de hoje utilizaremos um outro problema do Gym que é mais desafiador que o CartPole.


> **Atenção**: para entender melhor a tarefa leia a documentação do LunarLander disponível em http://gym.openai.com/envs/LunarLander-v2/.

Execute o código abaixo para visualizar alguns episódios do agente aleatório para ter uma melhor ideia da tarefa:

In [2]:
env = gym.make("LunarLander-v2")

agent = RandomAgent(env.observation_space, env.action_space, None)

utils.runner.evaluate(agent, env, n_episodes=10, render=True)

>> episode = 0 / 10, total_reward =  -159.5075, episode_length = 74
>> episode = 1 / 10, total_reward =  -215.9623, episode_length = 102
>> episode = 2 / 10, total_reward =  -116.4991, episode_length = 70
>> episode = 3 / 10, total_reward =  -101.6698, episode_length = 76
>> episode = 4 / 10, total_reward =     9.2598, episode_length = 77
>> episode = 5 / 10, total_reward =  -124.4620, episode_length = 96
>> episode = 6 / 10, total_reward =   -72.7559, episode_length = 124
>> episode = 7 / 10, total_reward =  -151.4637, episode_length = 103
>> episode = 8 / 10, total_reward =  -101.0856, episode_length = 70
>> episode = 9 / 10, total_reward =  -267.9343, episode_length = 86


## 1. Ambientes vetorizados no Gym

Pela primeira vez no curso estaremos utilizando ambientes vetorizados, isto é, que emulam o comportamente de vários ambientes sendo executados em paralelo.

Execute o código abaixo e tente entender como o ambiente retornado pelo `gym.vector.make` se comporta.

In [3]:
env = gym.vector.make("LunarLander-v2", num_envs=4, asynchronous=True)

print(env.processes)

print(env.observation_space, env.single_observation_space)
print(env.action_space, env.single_action_space)

[<ForkProcess(Worker<AsyncVectorEnv>-0, started daemon)>, <ForkProcess(Worker<AsyncVectorEnv>-1, started daemon)>, <ForkProcess(Worker<AsyncVectorEnv>-2, started daemon)>, <ForkProcess(Worker<AsyncVectorEnv>-3, started daemon)>]
Box(4, 8) Box(8,)
Tuple(Discrete(4), Discrete(4), Discrete(4), Discrete(4)) Discrete(4)


In [4]:
n_steps_per_env = 3

observations = env.reset()

step = 0

for _ in range(n_steps_per_env):
    actions = env.action_space.sample()
    observations, rewards, dones, _ = env.step(actions)
    step += len(observations)

    print(f">> step = {step}")
    print(f"observations =\n{observations}")
    print(f"actions = {actions}")
    print(f"rewards = {rewards}")
    print(f"dones = {dones}")
    print()
    
print(f">> num_envs = {env.num_envs}, n_steps_per_env = {n_steps_per_env}, total timesteps = {step}")

>> step = 4
observations =
[[ 9.9943159e-03  1.4090319e+00  5.1168972e-01 -5.4736994e-02
  -1.3448511e-02 -1.5427688e-01  0.0000000e+00  0.0000000e+00]
 [ 1.2449265e-03  1.4225172e+00  6.9121771e-02  2.4484667e-01
  -3.3980494e-03 -5.4628491e-02  0.0000000e+00  0.0000000e+00]
 [-9.8070148e-03  1.4327834e+00 -4.9022451e-01  4.9019155e-01
   1.2034555e-02  1.2543091e-01  0.0000000e+00  0.0000000e+00]
 [-3.1600953e-03  1.3925726e+00 -1.5505718e-01 -3.8886729e-01
   4.2763650e-03  4.7539223e-02  0.0000000e+00  0.0000000e+00]]
actions = (3, 3, 2, 2)
rewards = [-1.86480453  1.39528129 -1.7126427   4.22811332]
dones = [False False False False]

>> step = 8
observations =
[[ 0.01496611  1.4072112   0.50325024 -0.08099432 -0.01945782 -0.12019722
   0.          0.        ]
 [ 0.00175581  1.4287127   0.05460066  0.27534387 -0.00687592 -0.06956423
   0.          0.        ]
 [-0.01472578  1.4438038  -0.49785548  0.4897322   0.01794087  0.11813722
   0.          0.        ]
 [-0.00468664  1.3832222

## 2. Advantage Actor-Critic (A2C)

<img src="img/a2c-algo.png" alt="A2C Algorithm" style="width: 700px;"/>


> **Observação**: para uma introdução mais intuitiva do A2C recomendamos o blog post https://sudonull.com/post/32170-Intuitive-RL-Reinforcement-Learning-Introduction-to-Advantage-Actor-Critic-A2C. 

### 2.1 Compartilhando informação entre *Actor* e *Critic*: *2-head model*

Na arquitetura A2C é comum implementar o Actor e o Critic em um mesmo modelo que compartilha parâmetros. Execute o código abaixo e tente interpretar a figura.

In [5]:
num_envs = 8
env = gym.vector.make("LunarLander-v2", num_envs=num_envs, asynchronous=True)

In [6]:
config = {
    "actor_critic_net": {
        "hidden_layers": [64, 64],
        "activation": "tanh"
    },
}

In [7]:
actor_critic_net = build_actor_critic_network(env.single_observation_space, env.single_action_space, config["actor_critic_net"])
tf.keras.utils.plot_model(actor_critic_net, show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


### 2.2 Otimizando a política e a função Valor via *joint loss*

$$
[\theta, \phi] \leftarrow [\theta, \phi] + \alpha \nabla_{\theta, \phi}(L_{actor}(\theta) + L_{critic}(\phi))
$$

onde $L_{actor}(\theta)$ e $L_{critic}(\phi)$ correspondem respectivamente ao *policy loss* e *mean squared error*:
$$
\begin{align*}
L_{actor}(\theta) &= - \frac{1}{K} \sum_{t=1}^K \log \pi_{\theta}(\mathbf{a}_t|\mathbf{s}_t) \hat{A}_t^{(n)} \\
L_{critic}(\phi) &= \frac{1}{K} \sum_{t=1}^K  (V_{\phi}(\mathbf{s}_t) - \hat{R}_t)^2
\end{align*}
$$

> **<font color="red">IMPORTANTE</color>**: Note que para o problema de regressão que precisamos resolver para aprender o *critic*, o retorno descontado $\hat{R}_t$ funciona como o *target* e, portanto, deve ser considerado uma "constante" para o TensorFlow. No exercício abaixo você precisará se lembrar disso!



### 2.3  Bônus de exploração: entropia da distribuição de ações

Além do *joint loss* adicionaremos também um outro termo à função objetivo a fim de incentivar o agente a continuar explorando novas ações.

$$
H(\pi_{\theta}) = \mathbb{E}_{\mathbf{s}, \mathbf{a} \sim \sim \pi_{\theta}} \left[ - \log \pi_{\theta}(\mathbf{a}|\mathbf{s}) \right]
$$

Tente entender no código abaixo onde e como esse bônus da entropia é implementado.

## 3. Detalhes (importantes) de implementação

### 3.1 Truncando a norma dos gradientes

$$
\tilde\nabla L(\theta) = 
\begin{cases}
    \nabla L(\theta)                                     & \text{ se } \| \nabla L(\theta) \| < clip \\
    \frac{\nabla L(\theta)}{\| \nabla L(\theta) \|} clip & \text{ c.c. }
\end{cases}
$$

No código da classe abaixo, implementamos um truque conhecido em Deep Learning como *gradient clipping*. Ele consiste em preprocessar os gradientes de cada peso da rede truncando a sua magnitude caso ela passe de um certo limiar, ilustrado acima com o gradiente de uma função-objetivo $L(\theta)$ genérica. Esse truque é comum pois gradientes podem tomar magnitudes muito grandes quando o ambiente é ruidoso ou a rede muito grande. Isso gera instabilidades no treinamento e pode até levar pesos a valores inválidos como *Not a Number* (`Nan`). Implementamos esse preprocessamento logo antes de chamar `apply_gradients` no método `learn()` do agente.

### 3.2 *Learning rate scheduler*

$$
\theta \gets \theta - \alpha_t \nabla L(\theta)
$$

O segundo truque que implementamos abaixo é o uso de um agendador para a taxa de aprendizado (`learning rate`) $\alpha_t$. A equação acima ilustra a regra genérica de atualização dos parâmetros $\theta$ para alguma função objetivo $L(\theta)$. Por mais que otimizadores como Adam e RMSprop já façam uma escolha bem motivada de $\alpha_t$ de acordo com o valor base $\alpha$ passado ao construtor, foi observado empiricamente que diminuir $\alpha$ ao longo do treinamento ajuda a estabilizar o treinamento. Implementamos isso com o uso do `tf.keras.optimizers.schedules.PolynomialDecay` no lugar de um valor fixo para o `learning_rate` do RMSprop. Configuramos o *scheduler* de forma que $\alpha_t$ decresca linearmente até o final do aprendizado.

Intuitivamente, imagine que a política fique mais sensível a variações nos parâmetros $\theta$ com o passar do tempo. Para observar esse fenômeno, experimente passar um valor fixo para o `learning_rate` abaixo. É comum observar que, após um período inicial de melhora nas curvas de retorno acumulado, o desempenho cai (muitas vezes abruptamente) perto do final do treinamento.

### 3.3 Ponderando os componentes do *joint loss*

$$
L(\theta, \phi) = L_\text{actor} (\theta) + \beta L_\text{critic}(\phi) - \epsilon H(\phi)
$$

Outro aspecto crucial de implementação é a ponderação adequada da contribuição das *losses* de cada componente para o *joint loss*. De fato, é raro que as funções objetivo de *actor*, *critic* e entropia estejam na mesma escala: muitas vezes $L_\text{actor}(\theta)$ assume valores pequenos enquanto $L_\text{critic}(\phi)$, valores grandes, principalmente no início do treinamento quando a função-valor está mal-ajustada.

É fundamental então experimentar com coeficientes diferentes para os objetivos do *critic* e de entropia, implementados abaixo, respectivamente, com os campos `vf_loss_coeff` e `entropy_coeff` do `config`. Os valores padrão já foram ajustados, mas experimente mudá-los para atingir um desempenho melhor na tarefa.

---

**<font color="red">EXERCÍCIO-PROGRAMA:</font>**

Nesse exercício você deverá implementar o *value function loss* no método `_joint_loss_fn` da classe abaixo.


> **Nota 1**: consulte a documentação do MSE em https://www.tensorflow.org/api_docs/python/tf/keras/losses/MeanSquaredError. Será útil!

> **Nota 2**: você precisará utilizar a função `tf.stop_gradient` no cálculo do *target* na loss do Value Function. Isso permite que o TensorFlow não tente diferenciar o *target*. Veja nota <font color="red">IMPORTANTE</color> na Seção 2.2.

> **Nota 3**: se você tiver algum erro de "tipos", por exemplo, esperava-se `float32` em algum ponto do código mas o array do *NumPy* foi calculado como `float64` você poderá fazer *casting* manual de tipos usando o método de `astype("f")`.

In [8]:
class A2C(RLAgent):
    
    def __init__(self, obs_space, action_space, config):
        super().__init__(obs_space, action_space, config)
        
        self.memory = OnPolicyReplay()
        self.actor_critic = build_actor_critic_network(obs_space, action_space, config["actor_critic_net"])

        self.optimizer = tf.keras.optimizers.RMSprop(
            learning_rate=tf.keras.optimizers.schedules.PolynomialDecay(
                0.00083,
                decay_steps=config["total_timesteps"] / config["train_batch_size"],
                end_learning_rate=1e-4,
                power=1.0
            ))

    def act(self, obs):
        """
        Escolhe uma ação para ser tomada dada uma observação do ambiente.
        
        Args: 
            obs: observação do ambiente.
        
        Return:
            action: ação válida dentro do espaço de ações.
        """
        return self._act(obs).numpy()
        
    @tf.function
    def _act(self, obs):
        action_dist, _ = self.actor_critic(obs)
        return action_dist.sample()
    
    def observe(self, obs, action, reward, next_obs, done):
        """
        Registra na memória do agente uma transição do ambiente.

        Args:
            obs:            observação do ambiente antes da execução da ação.
            action:         ação escolhida pelo agente.
            reward (float): escalar indicando a recompensa obtida após a execução da ação.
            next_obs:       nova observação recebida do ambiente após a execução da ação.
            done (bool):    True se a nova observação corresponde a um estado terminal, False caso contrário.

        Return:
            None
        """
        self.memory.update(obs, action, reward, next_obs, done)

    def learn(self):
        """
        Método de treinamento do agente. A partir das experiências de sua memória,
        o agente aprende um novo comportamento.

        Args: 
            None

        Return:
            None
        """
        if self.memory.batch_size < self.config["train_batch_size"]:
            return
        
        batch = self.memory.sample()
        weights = self.actor_critic.trainable_weights

        with tf.GradientTape() as tape:
            policy_loss, vf_loss, entropy_loss = self._joint_loss_fn(batch)
            loss = policy_loss + self.config["vf_loss_coeff"] * tf.cast(vf_loss, tf.float32) - self.config["entropy_coeff"] * entropy_loss
            gradients = tape.gradient(loss, weights)
    
        gradients = tuple(tf.clip_by_norm(grad, clip_norm=0.5) for grad in gradients)
        self.optimizer.apply_gradients(zip(gradients, weights))
      
        return {
            "policy_loss": policy_loss.numpy(),
            "vf_loss": vf_loss.numpy(),
            "entropy_loss": entropy_loss.numpy()
        }

    def _joint_loss_fn(self, batch):
        """
        Calcula a função loss do policy gradients para um `batch` de transições.
        
        Um `batch` agrega arrays n-dimensionais. Cada array (e.g., batch["states"],
        batch["actions"], batch["rewards"]) tem como primeiras duas dimensões o número
        de passos dados no ambiente vetorizado e o número de ambientes em paralelo. 
        Por exemplo, batch["states"][t][k] devolve um array correspondendo ao estado 
        no passo t devolvido pelo k-ésimo ambiente.

        Args:
            batch (Dict[str, np.ndarray]): dicionário para acesso às matrizes de 
                estados, ações, recompensas, próximos estados e flags de terminação. 
        
        Return:
            loss (tf.Tensor): surrogate loss conjunta da política, função valor e
                bônus de entropia.
        """
        states = batch["states"]
        actions = batch["actions"]
        rewards = batch["rewards"]
        next_states = batch["next_states"]
        dones = batch["dones"]

        n_steps = len(states)
        gamma = self.config["gamma"]
        lambda_ = self.config["lambda"]

        action_dists, values = self.actor_critic(states)
        _, last_value = self.actor_critic(next_states[-1:])

        values = tf.squeeze(tf.concat([values, last_value], axis=0))
        values, next_values = values[:-1], values[1:]

        deltas = rewards + gamma * (1 - dones) * next_values - values

        returns = np.empty_like(rewards)
        advantages = np.empty_like(rewards)

        returns[-1] = rewards[-1] + gamma * (1 - dones[-1]) * next_values[-1]
        advantages[-1] = deltas[-1]

        for t in reversed(range(n_steps - 1)):
            returns[t] = rewards[t] + gamma * (1 - dones[t]) * returns[t+1]
            advantages[t] = deltas[t] + (gamma * lambda_) * (1 - dones[t]) * advantages[t+1]

        log_probs = action_dists.log_prob(actions)

        policy_loss = - tf.reduce_sum(log_probs * tf.stop_gradient(advantages.astype("f")))
        
        vf_criterion = tf.keras.losses.MeanSquaredError()
        vf_loss = vf_criterion(values, tf.stop_gradient(returns.astype("f")))

        entropy_loss = tf.reduce_mean(action_dists.entropy())

        return policy_loss, vf_loss, entropy_loss


Para treinar o seu agente A2C execute o código abaixo:

In [9]:
total_timesteps = 500_000

num_envs = 8
train_env = gym.vector.make("LunarLander-v2", num_envs=num_envs, asynchronous=True)

config = {
    "actor_critic_net": {
        "hidden_layers": [64, 64],
        "activation": "tanh"
    },
    "optimizer": {
        "class_name": "RMSprop",
        "config": {
            "learning_rate": 8e-4,
            "rho": 0.99
        }
    },
    "total_timesteps": total_timesteps,
    "train_batch_size": 40,
    "gamma": 0.995,
    "lambda": 1.0,
    "vf_loss_coeff": 0.25,
    "entropy_coeff": 1e-5
}

agent = A2C(train_env.single_observation_space, train_env.single_action_space, config)

timesteps, avg_total_rewards, losses = utils.runner.train(agent, train_env, total_timesteps)

plot_metrics(avg_total_rewards, losses)

Para visualizar o agente treinado execute o código abaixo:

In [10]:
eval_env = gym.make("LunarLander-v2")
utils.runner.evaluate(agent, eval_env, n_episodes=10, render=True)

>> episode = 0 / 10, total_reward =    48.8051, episode_length = 255
>> episode = 1 / 10, total_reward =    34.7169, episode_length = 241
>> episode = 2 / 10, total_reward =   206.4046, episode_length = 513
>> episode = 3 / 10, total_reward =   178.7825, episode_length = 553
>> episode = 4 / 10, total_reward =    19.7831, episode_length = 371
>> episode = 5 / 10, total_reward =   -31.1515, episode_length = 470
>> episode = 6 / 10, total_reward =    56.8251, episode_length = 253
>> episode = 7 / 10, total_reward =   230.3525, episode_length = 596
>> episode = 8 / 10, total_reward =    53.0976, episode_length = 256
>> episode = 9 / 10, total_reward =   -18.0218, episode_length = 430


---

**<font color="red">PARABÉNS!</font>**


Se você conseguiu chegar até aqui, parabéns! Você conseguiu fazer funcionar uma primeira versão do A2C. Sabemos que isso não é uma tarefa fácil... :)

Esperamos que você tenha se familiarizado com as principais ideias por trás de um algoritmo de Deep RL e tenha sentido um pouco na pele a dificuladade de treinar agentes de RL, mesmo para problemas aparentemente simples.